<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/stock_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
import torch
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm
import torch.nn.functional as F

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Microsoft_Stock.csv')

In [ ]:
data.head

In [ ]:
print(data.columns)

In [ ]:
#날짜데이터를 누락.
data.drop('Date', axis=1, inplace=True)

In [ ]:
data.columns

In [ ]:
#data 표준화.
scaler = StandardScaler()

std_data = scaler.fit_transform(data)

In [ ]:
std_data

In [ ]:
std_data[0,:]

In [ ]:
#단기모델을학습시키기위한 데이터 생성
#데이터는 연속한날짜 두개를 합침. 라벨은 종가를 기준으로 등락을 퍼센트로 구성.

label = data['Close'].to_numpy()

short_term_data = []
short_term_label = []

for i in range(len(std_data)-1):

    d = np.hstack([std_data[i,:], std_data[i+1,:]])
    p = (label[i+1]-label[i])/label[i]*100

    short_term_data.append(d)
    short_term_label.append(p)

short_term_data = torch.tensor(short_term_data).float()
short_term_label = torch.tensor(short_term_label).float()

print(short_term_data.shape)
print(short_term_label.shape)
print(short_term_data[0])
print(short_term_label[0])

In [ ]:
#데이터로더.
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

#인스턴스 생성.
custom_dataset = CustomDataset(short_term_data, short_term_label)

#데이터를 훈련 데이터와 검증 데이터, 테스트 데이터로 나누기. 파이토치데이터셋 인스턴스에 사용하면 라벨도 같이 분류됨.
#훈련세트 80%, 테스트세트 20% 분류
train_size = int(0.8 * len(short_term_data))
test_size = len(short_term_data) - train_size

train_dataset, test_dataset = random_split(custom_dataset, [train_size, test_size])

#테스트세트를 테스트세트 10%, 검증세트 10% 분류
test_size = int(0.5 * len(test_dataset))
val_size = len(test_dataset) - test_size

test_dataset, val_dataset = random_split(test_dataset, [test_size, val_size])

# 데이터 로더 생성.
# 데이터 로더 생성
batch_size = 2
train_DL = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_DL = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_DL = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# 데이터 로더를 통해 데이터 확인
for features, labels in train_DL:
    print(features)
    print(labels)
    break
# train_DL.dataset[0]

In [ ]:
print(len(train_DL.dataset))
print(len(test_DL.dataset))
print(len(val_DL.dataset))

In [ ]:
#단기모델(연속2일의 data를 받아 다음날의 주가상승률 예측)의 label생성
#주가가 지수적으로 증가하므로 증가율로 설정
class term1_model(nn.Module):
    def __init__(self):
        super().__init__()

        self.linear = nn.Sequential(nn.Linear(10,50), nn.ReLU(),
                                    nn.Linear(50,50), nn.ReLU(),
                                    nn.Linear(50,50), nn.ReLU(),
                                    nn.Linear(50,1))

    def forward(self, x):

        x = self.linear(x)

        return x

MSE (Mean Squared Error):

정의: 예측값과 실제값의 차이(오차)를 제곱한 값의 평균입니다.
특징: 오차의 제곱을 사용하기 때문에 큰 오차에 더 큰 벌점을 부여합니다. 이는 모델이 큰 오차를 내는 것을 특히 피하도록 유도합니다.
훈련에서의 사용: MSE는 그래디언트가 오차의 크기에 비례하기 때문에, 훈련 중에 큰 오차를 더 효과적으로 줄일 수 있습니다. 따라서 모델의 학습에 있어서 더 민감하고 강력한 피드백을 제공합니다.
MAE (Mean Absolute Error):

정의: 예측값과 실제값의 차이의 절대값의 평균입니다.
특징: 오차의 절대값을 사용하기 때문에, 모든 오차에 동일한 가중치를 부여합니다. 이는 큰 오차와 작은 오차를 동일하게 다룹니다.
성능 평가에서의 사용: MAE는 이상치에 대해 덜 민감하며, 오차의 '실제' 크기를 더 직관적으로 반영합니다. 따라서 모델의 성능을 사람이 이해하기 쉬운 방식으로 평가하는 데 유용합니다.

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def loss_epoch(model, DL, optimizer=None):
    num_samples = len(DL.dataset)
    running_loss = 0
    total_absolute_error = 0

    for x_batch, y_batch in tqdm(DL, leave=False):
        x_batch = x_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)

        # 모델 예측
        y_pred = model(x_batch)
        y_pred = y_pred.squeeze()

        # MSE 손실 계산
        loss = F.mse_loss(y_pred, y_batch)

        # 역전파와 최적화
        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # 손실과 MAE 업데이트
        running_loss += loss.item() * x_batch.size(0)
        total_absolute_error += torch.sum(torch.abs(y_pred - y_batch)).item()

    # 에포크 당 평균 손실과 MAE 계산
    epoch_loss = running_loss / num_samples
    epoch_mae = total_absolute_error / num_samples

    return epoch_loss, epoch_mae

In [ ]:
model = term1_model()
LR = 1e-3 # -1 인경우 한번에 0또는 1만 출력하는 로컬 미니멈으로 수렴.
optimizer = optim.Adam(model.parameters(), lr = LR)
loss_epoch(model, train_DL, optimizer=optimizer)

In [ ]:
def Train(model, train_DL, val_DL, optimizer, EPOCH):

    loss_history = {"train":[], "val":[]}
    acc_history = {"train":[], "val":[]}

    for ep in range(EPOCH):
        epoch_start = time.time()

        model.train() # train mode로 전환
        train_loss, train_acc, _ = loss_epoch(model, train_DL, optimizer)
        loss_history["train"] += [train_loss]
        acc_history["train"] += [train_acc]

        model.eval() # test mode로 전환
        with torch.no_grad():
            val_loss, val_acc, _ = loss_epoch(model, val_DL)
            loss_history["val"] += [val_loss]
            acc_history["val"] += [val_acc]

        # print loss
        print(f"train loss: {round(train_loss,5)}, "
              f"val loss: {round(val_loss,5)} \n"
              f"train acc: {round(train_acc,1)} %, "
              f"val acc: {round(val_acc,1)} %, time: {round(time.time()-epoch_start)} s")
        print("-"*20)

    return loss_history